# Preparation

In [1]:
# basic setup of the notebook
%pylab inline
from pprint import pprint # we use this to pretty print some stuff later
import numpy as np
from matplotlib.colors import LogNorm
from iminuit import Minuit
from iminuit import minimize
# from scipy.optimize import minimize
from scipy.special import expit

Populating the interactive namespace from numpy and matplotlib


In [2]:
# defining constants
mec_dim = 24  # dimension of matrix of rescaling
events_number = 1e6  # number of events in sample
cross_section = 1.19727e-39  # calculated cross section in .txt file
alpha = 0.3 # parameter of smoothness

delta_pT = np.array([75, 75, 100, 75, 75, 75, 75, 150, 150, 150, 250, 250, 1000])
delta_pT_plot = np.array([75, 75, 100, 75, 75, 75, 75, 150, 150, 150, 250, 250, 1000, 0])
delta_pL = np.array([500, 500, 500, 500, 500, 500, 500, 1000, 2000, 2000, 5000, 5000])
pT = np.array([75, 150, 250, 325, 400, 475, 550, 700, 850, 1000, 1250, 1500, 2500])
pT_plot = np.array([75, 150, 250, 325, 400, 475, 550, 700, 850, 1000, 1250, 1500, 2500, 3000])
pL = np.array([1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 6000, 8000, 10000, 15000, 20000])
bins_width_vanilla = np.kron(delta_pT, delta_pL)
bins_width = np.outer(bins_width_vanilla,  np.ones(mec_dim**2))

# print(bins_width)

In [3]:
# loading files
matrix_vanilla      = np.fromfile("data/matrix_LFG.dat",                sep=" ")  # distribution of MEC events in bins 
cross_nuwro         = np.fromfile("data/cros_total_nuwro_LFG.dat",      sep=" ")  # cross section of nuwro (w-o MEC)
cross_daniel        = np.fromfile("data/cros_total_daniel.dat",         sep=" ")  # cross section of MINERvA
covariance_vanilla  = np.fromfile("data/daniel_covariance_vanilla.dat", sep=" ")  # covariance matrix
cross_error_vanilla = np.fromfile("data/cros_error_daniel.dat",         sep=" ")  # error bars

In [4]:
# modifying files
matrix_unnormalized = np.reshape(matrix_vanilla,(156, mec_dim**2))  # reshaping MEC matrix
matrix = matrix_unnormalized * cross_section * 1e6 * 12 / 13 / bins_width / events_number  # normalizing matrix

covariance_noninverted = np.reshape(covariance_vanilla, (156,156))  # reshaping covarince matrix
covariance = pinv(covariance_noninverted)  # pseudo-inverse covariance matrix (used later); inverse doesnt exist

cross_error = np.where(cross_error_vanilla==0, 1, cross_error_vanilla)  # removing zeros from error bars

# rotation of results due to covariance matrix's look 
cross_daniel_cov = np.reshape(cross_daniel,(12,13)).transpose().flatten()  
cross_nuwro_cov = np.reshape(cross_nuwro,(12,13)).transpose().flatten()

In [5]:
def mysigmo(x): # my modified sigmoid
    return 10*expit(x*100-3)

#################################
# bounds and constrains
tmp = []
for i in range(0,24):  # fixing over diagonal elements
    tmp += [1,1] * i + [0.01, None] * (24-i)
    
bnd = np.reshape(tmp,(576,2))  # bounds - no zero in order to avoid division by it

##################################

def punishment(rescale): 

    upper = []
    for i in range(24):
        for j in range(0, i):
            upper += [24*i + j]  # elements' indexes over diagonal 

    lower = []
    for i in range(24):
        for j in range(i, 24):
            lower += [24*i + j]  # elements' indexes below diagonal 

    # d - diagonal_line 
    # r - right_line
    # b - bottom line
    # i - inner
    # . - both 
    #
    #     ---------
    #     |      .|
    #     |    dir|
    #     |  diiir|
    #     |.bbbbb.|
    #     ---------

    bottom_line    = range(24)
    right_line     = range(23, 24 ** 2, 24)
    diagonal_line  = [24*i + i for i in range(24)]
    inner          = [x for x in lower if x not in bottom_line and x not in right_line and x not in diagonal_line]

    pun = 0 # punishment

    for i in bottom_line[1:-1]:  # for bottom side of matrix
        pun += mysigmo(-rescale[i] / max( \
                rescale[i + 1], \
                rescale[i + 24],\
                rescale[i - 1]) + (1 - alpha))
        pun += mysigmo(rescale[i] / min( \
                rescale[i + 1], \
                rescale[i + 24],\
                rescale[i - 1]) - 1/(1 - alpha))

    for i in right_line[1:-1]:  # for right side of matrix
        pun += mysigmo(-rescale[i] / max( \
                rescale[i + 24], \
                rescale[i - 1],\
                rescale[i - 24]) + (1 - alpha))
        pun += mysigmo(rescale[i] / min( \
                rescale[i + 24], \
                rescale[i - 1],\
                rescale[i - 24]) - 1/(1 - alpha))

    for i in diagonal_line[1:-1]:  # for diagonal side of matrix
        pun += mysigmo(-rescale[i] / max( \
                rescale[i + 1], \
                rescale[i - 24]) + (1 - alpha))
        pun += mysigmo(rescale[i] / min( \
                rescale[i + 1], \
                rescale[i - 24]) - 1/(1 - alpha))

    for i in inner[0:-1]:  # for the rest 
        pun +=  mysigmo(-rescale[i] / max( \
                rescale[i + 1],\
                rescale[i - 1],\
                rescale[i + 24],\
                rescale[i - 24]) + (1 - alpha))
        pun += mysigmo(rescale[i] / min( \
                rescale[i + 1],\
                rescale[i - 1],\
                rescale[i + 24],\
                rescale[i - 24]) - 1/(1 - alpha))
        
    return pun

In [7]:
# preparation - useful function
def chi2_cov(rescaling, pun=True):  # definition of chi^2 with covariance
    mec = np.reshape(np.matmul(matrix, rescaling),(12,13)).flatten()  # rescaled MEC
        
    tmp = (cross_daniel_cov - cross_nuwro_cov - mec)
    if pun==True:
        return (np.matmul(np.matmul(tmp,covariance), tmp) + punishment(rescaling))  # chi^2 = tmp.cov.tmp
    else:
        return (np.matmul(np.matmul(tmp,covariance), tmp))

def starting_matrix(x):  # starting matrix - 1 above diagonal, x below diagonal
    tmp = []
    for i in range(0,24):
        tmp += [1] * i + [x + 0.1*random.random() for a in range(24-i)] # with random fluctuation
        
    return tmp

def plot_matrix(x, chi2, method, success):  # drawing matrix and saving it to .pdf file 
    # method = 1 -> scipy
    # method = 0 -> migrad
    rescale = np.reshape(x,(24,24))

    for i in range(24):
        for j in range(i + 1, 24):
            rescale[j][i] = None

    fig = plt.figure(num=1, figsize=(1, 1), dpi=1, facecolor='w', edgecolor='k')
    fig, ax = plt.subplots()

    for (j,i),label in np.ndenumerate(rescale):
        if (label>0):
            ax.text(50*i+25,50*j+25,"{0:.1f}".format(label),ha='center',va='center',fontsize=3,color="black",weight='bold')
        else:
            ax.text(50*i+25,50*j+25," ",ha='center',va='center',fontsize=3,color="white",weight='bold')

    im = ax.imshow(rescale, origin="lower",extent=(1,1200,1,1200))
    cbar = ax.figure.colorbar(im, ax=ax)
    cbar.ax.set_ylabel(" ", rotation=-90, va="bottom")
    
    ax.set_title('$\\chi^2$ =' + str('%.2f' % chi2) + \
             ', with constrains $\\alpha = $' + str(alpha) + \
             ",\n method - " + str(method) + \
             ", success - " + str(success), fontsize=10)
    
    savetxt("Minuit/matrices/matrix_" + str('%.2f' % chi2) + "_" + str(method) + ".dat", rescale)
    fig.savefig("Minuit/plots/matrix_" + str('%.2f' % chi2) + "_" + str(method) + ".pdf",dpi=300, bbox_inches='tight', pad_inches=0, quality=95)

    plt.show()
    
def plot_results(x=np.ones(mec_dim**2), title="", save=True):
    mec = np.reshape(np.matmul(matrix, x),(13,12)).transpose().flatten()  # MEC cross section
    mec_org = np.reshape(np.matmul(matrix, np.ones(mec_dim**2)),(13,12)).transpose().flatten()  # MEC cross section

    for i in range(12):
        plt.figure(i)
        
        kosz = i*13
        tmp1 = np.append(cross_nuwro[kosz:kosz+13],cross_nuwro[kosz+12])
        tmp2 = np.append(mec[kosz:kosz+13],mec[kosz+12])
        tmp3 = np.append(mec_org[kosz:kosz+13],mec_org[kosz+12])
        
        plt.step(pT_plot-delta_pT_plot/2, 
                 tmp1+tmp2,
                 where="post",
                 label="after rescaling")
        plt.step(pT_plot-delta_pT_plot/2, 
                 tmp1+tmp3,
                 where="post",
                 label="before rescaling")
        plt.errorbar(pT, 
                     cross_daniel[kosz:kosz+13], 
                     yerr=cross_error_vanilla[kosz:kosz+13], 
                     xerr=0, 
                     fmt="r+")
        
        plt.title(title + " ; " + str(pL[i]) + " < $p_L$ < " + str(pL[i+1]))
        
        leg = plt.legend()
        
        xlabel("$p_T$ [MeV]")
        ylabel("$\\sigma$ [cm$^2$ \\ GeV$^2$ \\ nucleon]")
        
#         xlim(0, None)
#         ylim(0, None)
        
        if save:
            plt.savefig("Minuit/plots/" + str(title) + "_matrix_" + str(i) + ".pdf")

# migrad()

In [ ]:
# test_1 - chi^2 w-o covariance
# chi2 = (((cross_daniel - cross_nuwro - mec)/cross_error)**2).sum()
# print(chi2)

# print(chi2_cov(np.ones(576)))

In [ ]:
# # old method - using Minuit interface -> it easier to use minimize fuction than migrad
# non_negative = np.reshape((0, None) * mec_dim**2, (mec_dim**2,2))  # every element should not be negative

# triangular = []
# for i in range(0,24):  # fixing over diagonal elements
#     triangular += [True] * i + [False] * (24-i)
    
# def m(x):  # m for different starting parameter
#     return Minuit.from_array_func(chi2_cov, 
#                                   starting_matrix(x), 
#                                   error=0.1, 
#                                   errordef=1, 
#                                   limit=non_negative, 
#                                   fix=triangular,
#                                   use_array_call=True)

In [ ]:
# # ploting one solution -- old method
# tmp = m(1)
# tmp.migrad()
# plot_matrix(tmp.np_values(), tmp.fval, "migrad", str(tmp.migrad_ok()))

In [ ]:
# checking Minuit -- easier method

res = minimize(chi2_cov, 
               res["x"], 
               bounds=bnd,
               options={"maxfev":1000000})

plot_matrix(res["x"], chi2_cov(res["x"], False) , "migrad", res["success"])

In [ ]:
# plot_results(x=np.array(res["x"]).flatten(), title="MINUIT2", save=True)

In [ ]:
# # how does the result change with respect to starting matrix
# chi2_of_starting_param = []
# starting_param = []

# for i in numpy.arange(0, 1, 0.1):
#     tmp = m(i)
#     tmp.migrad()
#     chi2_of_starting_param += [tmp.fval]
#     starting_param += [i]
    
# xlabel("param of starting matrix")
# ylabel("$\\chi^2$")
# plot(starting_param, chi2_of_starting_param,".")
# # savefig("/home/michaszko/Desktop/lol.pdf")

In [ ]:
# # how does the result osccilate 
# chi2_of_starting_param = []
# starting_param = []

# for i in range(10):
#     tmp = m(0.1)
#     tmp.migrad()
#     chi2_of_starting_param += [tmp.fval]
#     starting_param += [i]
    
# xlabel("param of starting matrix")
# ylabel("$\\chi^2$")
# plot(starting_param, chi2_of_starting_param,".")
# # savefig("/home/michaszko/Desktop/lol.pdf")

# minimize() 
### with scipy